Text Generator

In [36]:
import tensorflow as tf
import numpy as np
import os
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from string import punctuation

In [14]:
# load ascii text and covert to lowercase
import requests
filename = "samples.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [15]:
sequence_length = 100
BATCH_SIZE = 128
EPOCHS = 30
# dataset file path
FILE_PATH = "sample1.txt"
BASENAME = os.path.basename(FILE_PATH)
# read the data
text = open(FILE_PATH, encoding="utf-8").read()
# remove caps, comment this code if you want uppercase characters as well
text = text.lower()
# remove punctuation
text = text.translate(str.maketrans("", "", punctuation))

In [16]:
# print some stats
n_chars = len(text)
vocab = ''.join(sorted(set(text)))
print("unique_chars:", vocab)
n_unique_chars = len(vocab)
print("Number of characters:", n_chars)
print("Number of unique characters:", n_unique_chars)

unique_chars: 
 0123456789abcdefghijklmnopqrstuvwxyz﻿
Number of characters: 158596
Number of unique characters: 39


In [17]:
# dictionary that converts characters to integers
char2int = {c: i for i, c in enumerate(vocab)}
# dictionary that converts integers to characters
int2char = {i: c for i, c in enumerate(vocab)}

In [18]:
# save these dictionaries for later generation
pickle.dump(char2int, open(f"{BASENAME}-char2int.pickle", "wb"))
pickle.dump(int2char, open(f"{BASENAME}-int2char.pickle", "wb"))

In [19]:
# convert all text into integers
encoded_text = np.array([char2int[c] for c in text])

In [20]:
# construct tf.data.Dataset object
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [21]:
# print first 5 characters
for char in char_dataset.take(8):
    print(char.numpy(), int2char[char.numpy()])

38 ﻿
27 p
29 r
26 o
21 j
16 e
14 c
31 t


In [22]:
# build sequences by batching
sequences = char_dataset.batch(2*sequence_length + 1, drop_remainder=True)

# print sequences
for sequence in sequences.take(2):
    print(''.join([int2char[i] for i in sequence.numpy()]))

﻿project gutenbergs alices adventures in wonderland by lewis carroll



this ebook is for the use of anyone anywhere at no cost and with

almost no restrictions whatsoever  you may copy it give it away
 or

reuse it under the terms of the project gutenberg license included

with this ebook or online at wwwgutenbergorg





title alices adventures in wonderland



author lewis carroll



posting date 


In [23]:
def split_sample(sample):
    # example :
    # sequence_length is 10
    # sample is "python is a great pro" (21 length)
    # ds will equal to ('python is ', 'a') encoded as integers
    ds = tf.data.Dataset.from_tensors((sample[:sequence_length], sample[sequence_length]))
    for i in range(1, (len(sample)-1) // 2):
        # first (input_, target) will be ('ython is a', ' ')
        # second (input_, target) will be ('thon is a ', 'g')
        # third (input_, target) will be ('hon is a g', 'r')
        # and so on
        input_ = sample[i: i+sequence_length]
        target = sample[i+sequence_length]
        # extend the dataset with these samples by concatenate() method
        other_ds = tf.data.Dataset.from_tensors((input_, target))
        ds = ds.concatenate(other_ds)
    return ds

# prepare inputs and targets
dataset = sequences.flat_map(split_sample)

In [24]:
def one_hot_samples(input_, target):
    # onehot encode the inputs and the targets
    # Example:
    # if character 'd' is encoded as 3 and n_unique_chars = 5
    # result should be the vector: [0, 0, 0, 1, 0], since 'd' is the 4th character
    return tf.one_hot(input_, n_unique_chars), tf.one_hot(target, n_unique_chars)

dataset = dataset.map(one_hot_samples)

In [25]:
# print first 2 samples
for element in dataset.take(2):
    print("Input:", ''.join([int2char[np.argmax(char_vector)] for char_vector in element[0].numpy()]))
    print("Target:", int2char[np.argmax(element[1].numpy())])
    print("Input shape:", element[0].shape)
    print("Target shape:", element[1].shape)
    print("="*50, "\n")

Input: ﻿project gutenbergs alices adventures in wonderland by lewis carroll



this ebook is for the use of
Target:  
Input shape: (100, 39)
Target shape: (39,)

Input: project gutenbergs alices adventures in wonderland by lewis carroll



this ebook is for the use of 
Target: a
Input shape: (100, 39)
Target shape: (39,)



In [26]:
# repeat, shuffle and batch the dataset
ds = dataset.repeat().shuffle(1024).batch(BATCH_SIZE, drop_remainder=True)

In [27]:
model = Sequential([
    LSTM(256, input_shape=(sequence_length, n_unique_chars), return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(n_unique_chars, activation="softmax"),
])

In [28]:
# define the model path
model_weights_path = f"results/{BASENAME}-{sequence_length}.h5"
model.summary()
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 256)          303104    
                                                                 
 dropout (Dropout)           (None, 100, 256)          0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 39)                10023     
                                                                 
Total params: 838,439
Trainable params: 838,439
Non-trainable params: 0
_________________________________________________________________


In [29]:
# make results folder if does not exist yet
if not os.path.isdir("results"):
    os.mkdir("results")
# train the model
model.fit(ds, steps_per_epoch=(len(encoded_text) - sequence_length) // BATCH_SIZE, epochs=EPOCHS)
# save the model
model.save(model_weights_path)

Epoch 1/30
1238/1238 [==============================] - 1657s 1s/step - loss: 2.2210 - accuracy: 0.3586
Epoch 2/30
1238/1238 [==============================] - 1663s 1s/step - loss: 1.7236 - accuracy: 0.4872
Epoch 3/30
1238/1238 [==============================] - 1596s 1s/step - loss: 1.5256 - accuracy: 0.5403
Epoch 4/30
1238/1238 [==============================] - 1557s 1s/step - loss: 1.3823 - accuracy: 0.5801
Epoch 5/30
1238/1238 [==============================] - 1530s 1s/step - loss: 1.2611 - accuracy: 0.6126
Epoch 6/30
1238/1238 [==============================] - 1463s 1s/step - loss: 1.1515 - accuracy: 0.6433
Epoch 7/30
1238/1238 [==============================] - 1378s 1s/step - loss: 1.0458 - accuracy: 0.6731
Epoch 8/30
1238/1238 [==============================] - 1320s 1s/step - loss: 0.9551 - accuracy: 0.6980
Epoch 9/30
1238/1238 [==============================] - 1223s 988ms/step - loss: 0.8700 - accuracy: 0.7240
Epoch 10/30
1238/1238 [==============================] - 1216

In [30]:
import numpy as np
import pickle
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Activation
import os

sequence_length = 100
# dataset file path
FILE_PATH = "sample1.txt"
# FILE_PATH = "data/python_code.py"
BASENAME = os.path.basename(FILE_PATH)

In [31]:
seed = "chapter xiii"

In [32]:
# load vocab dictionaries
char2int = pickle.load(open(f"{BASENAME}-char2int.pickle", "rb"))
int2char = pickle.load(open(f"{BASENAME}-int2char.pickle", "rb"))
vocab_size = len(char2int)

In [33]:
# building the model
model = Sequential([
    LSTM(256, input_shape=(sequence_length, vocab_size), return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(vocab_size, activation="softmax"),
])

In [34]:
# load the optimal weights
model.load_weights(f"results/{BASENAME}-{sequence_length}.h5")

In [35]:
s = seed
n_chars = 400
# generate 400 characters
generated = ""
for i in tqdm.tqdm(range(n_chars), "Generating text"):
    # make the input sequence
    X = np.zeros((1, sequence_length, vocab_size))
    for t, char in enumerate(seed):
        X[0, (sequence_length - len(seed)) + t, char2int[char]] = 1
    # predict the next character
    predicted = model.predict(X, verbose=0)[0]
    # converting the vector to an integer
    next_index = np.argmax(predicted)
    # converting the integer to a character
    next_char = int2char[next_index]
    # add the character to results
    generated += next_char
    # shift seed and the predicted character
    seed = seed[1:] + next_char

print("Seed:", s)
print("Generated text:")
print(generated)

Generating text: 100%|███████████████████████████████████████████████████████████████| 400/400 [00:21<00:00, 18.18it/s]

Seed: chapter xiii
Generated text:
 aloce she went on in

the pat open out im to a dony wrote i wonder which way to find in a more

hop question the gryphon and the fan and garden and make out the grow of alice and wondering why i dont get out

a tream seall

beaster alice as she remembered

of minute or two she began on by paristing the branch of her however it was good rupbis do and be read



and with to its generally

offended 
